In [11]:
from modeler.modeler import Modeler as m
from database.market import Market
from database.strategy import Strategy
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from sklearn.metrics import r2_score, mean_absolute_percentage_error

In [2]:
market = Market()
strategy = Strategy()

In [6]:
strategy.connect()
weekly_sim = strategy.retrieve("weekly_sim")
quarterly_sim = strategy.retrieve("quarterly_sim")
strategy.disconnect()
market.connect()
prices = market.retrieve("prices")
market.disconnect()

In [7]:
prices["date"] = pd.to_datetime(prices["date"])
prices["date"]
prices["year"] = [x.year for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]

In [10]:
qp = prices.groupby(["year","quarter","ticker"]).mean().reset_index()[["year","quarter","ticker","adjClose"]]
wp = prices.groupby(["year","week","ticker"]).mean().reset_index()[["year","week","ticker","adjClose"]]
quarterly_set = quarterly_sim.merge(qp,on=["year","quarter","ticker"],how="left")
weekly_set = weekly_sim.merge(wp,on=["year","week","ticker"],how="left")

In [24]:
weekly_set

,_id,year,quarter,week,ticker,weekly_price_regression_prediction,score,cty,mty,tw,adjClose
0,615922544f23cb4b0f391759,2018,1,1,JNPR,25.731917,0.999608,1,1,14,25.401783
1,615922544f23cb4b0f39175a,2018,1,1,MS,47.897650,0.999608,1,1,14,45.648959
2,615922544f23cb4b0f39175b,2018,1,1,BA,282.879693,0.999608,1,1,14,291.380636
3,615922544f23cb4b0f39175c,2018,1,1,FBHS,65.417754,0.999608,1,1,14,60.416694
4,615922544f23cb4b0f39175d,2018,1,1,T,30.826135,0.999608,1,1,14,28.768766
...,...,...,...,...,...,...,...,...,...,...,...
3530321,615a674236adaf1db6dc7a20,2018,4,48,WMB,17.866766,0.397559,2,1,28,20.396689
3530322,615a674236adaf1db6dc7a21,2018,4,49,WMB,18.346062,0.397559,2,1,28,20.183043
3530323,615a674236adaf1db6dc7a22,2018,4,50,WMB,17.946297,0.397559,2,1,28,19.452142
3530324,615a674236adaf1db6dc7a23,2018,4,51,WMB,16.578518,0.397559,2,1,28,18.277481


In [17]:
##quarterly_analysis
qa = []
for cty in range(1,5):
    for mty in range(3,6):
        try:
            current_set = quarterly_set[(quarterly_set["category_training_year"]==cty) & (quarterly_set["model_training_year"]==mty)]
            current_set["actual"] = current_set["adjClose"].shift(-1)
            current_set.dropna(inplace=True)
            r2 = r2_score(current_set["quarterly_price_regression_prediction"],current_set["adjClose"])
            mape = mean_absolute_percentage_error(current_set["quarterly_price_regression_prediction"],current_set["adjClose"])
            qa.append({
                "cty":cty,
                "mty":mty,
                "r2":r2,
                "mape":mape
            })
        except Exception as e:
            print(str(e))
hello = pd.DataFrame(qa)
hello["score"] = hello["r2"] * 0.5 + (1-hello["mape"])*0.5
hello.sort_values("score",ascending=False)

In [26]:
##quarterly_analysis
qa = []
for cty in range(1,5):
    for mty in range(3,6):
        try:
            current_set = quarterly_set[(quarterly_set["category_training_year"]==cty) & (quarterly_set["model_training_year"]==mty)]
            current_set["actual"] = current_set["adjClose"].shift(-1)
            current_set.dropna(inplace=True)
            r2 = r2_score(current_set["quarterly_price_regression_prediction"],current_set["adjClose"])
            mape = mean_absolute_percentage_error(current_set["quarterly_price_regression_prediction"],current_set["adjClose"])
            qa.append({
                "cty":cty,
                "mty":mty,
                "r2":r2,
                "mape":mape
            })
        except Exception as e:
            print(str(e))
hello = pd.DataFrame(qa)
hello["score"] = hello["r2"] * 0.5 + (1-hello["mape"])*0.5
hello.sort_values("score",ascending=False)

,cty,mty,r2,mape,score
0,1,3,0.820909,0.368277,0.726316
6,3,3,0.845291,0.413604,0.715843
1,1,4,0.759846,0.343746,0.708050
2,1,5,0.662770,0.290819,0.685976
11,4,5,0.758846,0.391195,0.683825
8,3,5,0.833328,0.507657,0.662836
5,2,5,0.638064,0.375102,0.631481
9,4,3,0.649129,0.405349,0.621890
10,4,4,0.613659,0.384122,0.614768
4,2,4,0.643510,0.626732,0.508389


In [29]:
##quarterly_analysis
qa = []
for tw in range(14,68,14):
    for cty in range(1,5):
        for mty in range(1,6):
            try:
                current_set = weekly_set[(weekly_set["cty"]==cty) & (weekly_set["mty"]==mty) & (weekly_set["tw"]==tw)]
                if current_set.index.size > 0:
                    current_set["actual"] = current_set["adjClose"].shift(-1)
                    current_set.dropna(inplace=True)
                    r2 = r2_score(current_set["weekly_price_regression_prediction"],current_set["adjClose"])
                    mape = mean_absolute_percentage_error(current_set["weekly_price_regression_prediction"],current_set["adjClose"])
                    qa.append({
                        "cty":cty,
                        "mty":mty,
                        "tw":tw,
                        "r2":r2,
                        "mape":mape
                    })
            except Exception as e:
                print(str(e))
hello = pd.DataFrame(qa)
hello["score"] = hello["r2"] * 0.5 + (1-hello["mape"])*0.5
hello.sort_values("score",ascending=False)

,cty,mty,tw,r2,mape,score
4,1,5,14,0.971221,0.039533,0.965844
32,4,1,28,0.974447,0.049688,0.962379
8,2,4,14,0.957368,0.043255,0.957057
27,2,4,28,0.957366,0.043262,0.957052
14,3,5,14,0.959228,0.045650,0.956789
26,2,3,28,0.950543,0.045345,0.952599
7,2,3,14,0.950534,0.045757,0.952388
22,1,3,28,0.947588,0.046099,0.950744
34,4,3,28,0.943629,0.048745,0.947442
23,1,4,28,0.934899,0.043897,0.945501
